In [2]:
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, make_scorer, classification_report
import pandas as pd
import numpy as np
import random

In [3]:
def preprocessing_1(X):
    X_normalized = []
    for i in range(len(X)):
        X_normalized.append(np.array(X[i])/sum(X[i]))
    return X_normalized

In [4]:
def save_pickle(file_name, file_path):
    with open(file_path, 'wb') as fp:
        pickle.dump(file_name, fp)

In [5]:
def remove_nan(npi_indigo_spl):
    npi_indigo_spl_non_nan = {}
    for npi in npi_indigo_spl:
        if isinstance(npi_indigo_spl[npi], str):
            npi_indigo_spl_non_nan[npi] = npi_indigo_spl[npi]
    return npi_indigo_spl_non_nan

In [6]:
def create_dataset(npi_distributions, npi_indigo_spl):
    dataset = {'NPI':[], 'features':[], 'labels':[]}
    for npi in npi_distributions:
        if int(npi) in npi_indigo_spl:
            dataset['NPI'].append(npi)
            dataset['features'].append(npi_distributions[npi])
            dataset['labels'].append(npi_indigo_spl[int(npi)])
    return dataset

In [7]:
def get_less_than_threshold_categories(y, threshold):
    labels_count = {}
    for lab in y:
        if lab in labels_count:
            labels_count[lab] += 1
        else:
            labels_count[lab] = 1

    useless = []
    for lab in labels_count:
        if labels_count[lab] < threshold:
            useless.append(lab)
    return useless

In [8]:
def remove_useless(useless, y, X):
    for i in reversed(range(len(y))):
        if y[i] in useless:
            del y[i]
            del X[i]
    return X, y

In [9]:
#def encode_labels(y):
#    label_encoder = LabelEncoder()
#    return label_encoder.fit_transform(y), label_encoder

In [10]:
def sanity_check(y_train):
    check = set(y_train)

    print('Max value in input: ' +  str(max(check)))
    print('"length-1" of set of input: ' + str(len(check)-1))
    assert max(check) == (len(check)-1)

In [11]:
def remove_from_y(X_train_, y_train_, remove_list):
    
    X_train_rm = []
    y_train_rm = []

    for i in range(len(X_train_)):
        if y_train_[i] not in remove_list:
            X_train_rm.append(X_train_[i])
            y_train_rm.append(y_train_[i])
    return X_train_rm, y_train_rm

In [12]:
def check_dataset_distribution(y_train, k):
    dataset_distribution = {}
    total = 0
    for i in range(len(y_train)):
        if y_train[i] not in dataset_distribution:
            dataset_distribution[y_train[i]] = 1
        else:
            dataset_distribution[y_train[i]] += 1
        total += 1
    
    print(dataset_distribution)
    remove_list = []
    for key in dataset_distribution:
        if dataset_distribution[key] < k:
            remove_list.append(key)
    
    print(list(dataset_distribution.values()))
    print(max(list(dataset_distribution.values())))
    print(min(list(dataset_distribution.values())))
    print(np.std(list(dataset_distribution.values())))
    return remove_list

In [13]:
def encode_labels_2way(y):
    encoded_y = []
    for l in y:
        if '-Surgery' in l or '-Minor' in l:
            encoded_y.append(0)
        else:
            encoded_y.append(1)
    return encoded_y

In [14]:
def encode_labels_2way_custom(y, label):
    encoded_y = []
    for l in y:
        if label in l:
            encoded_y.append(0)
        else:
            encoded_y.append(1)
    return encoded_y

In [15]:
def overpopulate(X, y, op_multiplier, op_ratio_retained):
    X_oped = []
    y_oped = []

    for i in range(len(y)):
        if y[i] == 0:
            for _ in range(op_multiplier):
                X_oped.append(X[i])
                y_oped.append(y[i])
        elif random.uniform(0, 1) < op_ratio_retained:
            X_oped.append(X[i])
            y_oped.append(y[i])
    
    return X_oped, y_oped

In [16]:
def get_top_k_cpts_with_multipliers(normalized_distributions, k):
    
    all_cpt_pos = {}
    for i in range(len(normalized_distributions)):
        cur_top_agg = normalized_distributions[i].argsort()[-k:][::-1]
        for pos in cur_top_agg:
            if pos not in all_cpt_pos:
                all_cpt_pos[pos] = 0
            all_cpt_pos[pos] += 1
                
    for cpt_pos in all_cpt_pos:
        all_cpt_pos[cpt_pos] = np.log(len(normalized_distributions)/all_cpt_pos[cpt_pos])
    
    return all_cpt_pos

In [17]:
def get_top_k_cpts(normalized_distributions, k, tfidf = True):
    
    all_cpt_pos = {}
    # print("Printing CPT codes with speciality")
    for i in range(len(normalized_distributions)):
        cur_top_agg = normalized_distributions[i].argsort()[-k:][::-1]
        for pos in cur_top_agg:
            if pos not in all_cpt_pos:
                all_cpt_pos[pos] = 0
            all_cpt_pos[pos] += 1
    
    # single = [code for code in all_cpt_pos if all_cpt_pos[code] == 1]; double = [code for code in all_cpt_pos if all_cpt_pos[code] == 2]
    # print(single)
    # print(double)
    # print(len(double) + len(single))

    if tfidf:
        for cpt_pos in all_cpt_pos:
            all_cpt_pos[cpt_pos] = np.log(len(normalized_distributions)/all_cpt_pos[cpt_pos])

        
    return all_cpt_pos

In [18]:
def get_idf_vector(all_cpt_pos, num_featurs):
    idf_vector = []
    for i in range(num_featurs):
        if i in all_cpt_pos:
            idf_vector.append(all_cpt_pos[i])
        else:
            idf_vector.append(0)

    return idf_vector

In [19]:
def normalize(X):
    for i in range(len(X)):
        X[i] = X[i]/sum(X[i])
    return X

In [20]:
def get_distributions(X, y):
    distributions = [np.array(len(X[0])*[0.0]) for _ in range(len(set(y)))]
    for i in range(len(X)):
        # if i%10000 == 0:
        #     print("Done: " + str(i))
        distributions[y[i]] += X[i]
    
    distributions[1] += distributions[0]
    
    distributions = normalize(distributions)

    return distributions

In [21]:
def get_tfidf(tf_matrix, idf_vector):
    tfidfs = []
    for tf in tf_matrix:
        tfidfs.append(np.multiply(tf, idf_vector[0]))
    return tfidfs

In [22]:
def reduce_dimention(all_cpt_idfs, X_normalized):
    cpt_pos = list(all_cpt_idfs.keys())
    #print(cpt_pos)

    X_reduced = []

    for distribution in X_normalized:
        temp = []
        for pos in cpt_pos:
            if all_cpt_idfs[pos] >= 0.001:
                temp.append(distribution[pos])
        X_reduced.append(temp)
    
    return X_reduced

In [23]:
npi_features_p1 = pd.read_pickle('./chuncked_npi_ncpcs_2019_0_.pkl')
npi_features_adder = pd.read_pickle('./concerned_specialities_npi_features9.pkl')

In [24]:
npi_features = npi_features_p1 | npi_features_adder

In [25]:
npi_indigo_spl = pd.read_pickle('./npi_indigo_spl.pkl')
npi_indigo_spl_non_nan = remove_nan(npi_indigo_spl)

In [26]:
dataset = create_dataset(npi_features, npi_indigo_spl_non_nan)

In [27]:
X = dataset['features']
y = dataset['labels']

In [28]:
print(len(X))
print(len(y))

192345
192345


Note: Maybe try a binary model of seperating surgical and non-surgical doctors and then create further indigo speciality distinctions for each of these 2 classes.

Note: Create a binary model each indigo speciality (True / False). Kind of a multi-label classification model.

In [29]:
def get_results(random_search, results_test, results_train, X_val, y_val, X_train, y_train, spl, k):
    predicted_probabilities = random_search.best_estimator_.predict_proba(X_val)
    predictions_test = np.argmax(predicted_probabilities, axis=1)

    predicted_probabilities = random_search.best_estimator_.predict_proba(X_train)
    predictions_train = np.argmax(predicted_probabilities, axis=1)

    test_out = classification_report(y_val, predictions_test, target_names=[spl, 'Others'], output_dict=True)
    train_out = classification_report(y_train, predictions_train, target_names=[spl, 'Others'], output_dict=True)

    results_train.append(["CPT top " + str(k) + " codes", "Train", spl, train_out['Others']['precision'], train_out['Others']['recall'], train_out[spl]['precision'], train_out[spl]['recall'], train_out['macro avg']['f1-score'], len(y_train[y_train==0]), len(y_train[y_train==1]), random_search.best_params_])
    results_test.append(["CPT top " + str(k) + " codes", "Test", spl, test_out['Others']['precision'], test_out['Others']['recall'], test_out[spl]['precision'], test_out[spl]['recall'], test_out['macro avg']['f1-score'], len(y_val[y_val==0]), len(y_val[y_val==1]), random_search.best_params_])

    return results_train, results_test

In [30]:
def spit_train_test_split(X, y, speciality, param_dist, results_test, results_train, k=25, tfidf=True, runs_per_speciality_per_k=3, majority_class_data_split=0.1, minority_class_op_coef=1, test_size=0.3, random_state=897):
    encoded_y = encode_labels_2way_custom(y, speciality)
    encoded_y_np = np.array(encoded_y)

    if len(encoded_y_np[encoded_y_np==0]) > 0:

        if len(encoded_y_np[encoded_y_np==0]) > 1500:
            runs_per_speciality_per_k = 1
        
        for _ in range(runs_per_speciality_per_k):

            X_oped, y_oped = overpopulate(X, encoded_y, minority_class_op_coef, majority_class_data_split)
            X_oped_normalized = preprocessing_1(X_oped)

            normalized_class_distributions = get_distributions(X_oped_normalized, y_oped)
            all_cpt_pos_idf = get_top_k_cpts(normalized_class_distributions, k, tfidf)
            idf_vector = get_idf_vector(all_cpt_pos_idf, len(X_oped_normalized[0]))

            X_oped_normalized_reduced = reduce_dimention(all_cpt_pos_idf, X_oped_normalized)
            # print(np.array(X_oped_normalized_reduced).shape)
            idf_vector_reduced = reduce_dimention(all_cpt_pos_idf, [idf_vector])
            # print(idf_vector_reduced)

            X_oped_normalized_reduced_tfidfs = get_tfidf(X_oped_normalized_reduced, idf_vector_reduced)
            X_oped_normalized_reduced_tfidfs = np.array(X_oped_normalized_reduced_tfidfs)
            
            X_train, X_val, y_train, y_val = train_test_split(X_oped_normalized_reduced_tfidfs, y_oped, test_size=test_size, random_state=random_state, stratify=y_oped, shuffle=True)
            # X_train, y_train = overpopulate(X_train, y_train, minority_class_op_coef, 1)
            X_train = np.array(X_train)
            y_train = np.array(y_train)

            # print(X_train.shape)
            y_train = np.array(y_train)
            # print(len(y_train[y_train==0]), len(y_train[y_train==1]))
            y_val = np.array(y_val)
            # print(len(y_val[y_val==0]), len(y_val[y_val==1]))

            model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(set(y_train)), tree_method='gpu_hist', gpu_id=0)
            random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=4, scoring='accuracy', n_jobs=-1, cv=2, verbose=3, random_state=53)
            random_search.fit(X_train, y_train)
            # print("Best Estimator: ", random_search.best_params_)
            results_train, results_test = get_results(random_search, results_test, results_train, X_val, y_val, X_train, y_train, speciality, k)
            model = None
            random_search = None
        results_train.append(len(results_train[0])*[""])
        results_test.append(len(results_test[0])*[""])
        # print('\033[1m' + '\033[92m' + "Done: " + speciality + '\033[0m' + '\033[0m')
    else:
        print('\033[1m' + '\033[91m' + speciality + " not in CPT top k codes" + '\033[0m' + '\033[0m')
    
    return results_train, results_test


In [32]:
# specialities_lis = pd.read_pickle("./specialities_lis.pkl")
# len(specialities_lis)
# specialities_lis = specialities_lis[50:]
# len(specialities_lis)
specialities_lis = [
    "Clinical Nurse Specialist",
    "Certified Nurse Midwife (CNM)",
    "Anesthesiology Assistant (AA)",
    "Cardiovascular Disease-Surgery",
    "Vascular-Surgery",
    "Derma tology-Minor Surgery",
    "Neurology-Surgery",
    "Gynecology-Minor Surgery",
    "Gynecology-No Surgery",
    "General Preventive Med-No Surgery",
    "Orthopedic Incl Back-Surgery",
    "Ophthalmology-Minor Surgery",
    "Ophthalmology-Surgery",
    "Otorhinolaryngology-No Surgery",
    "Otorhinolaryngology-Surgery",
    "Radiology Diagnostic-Minor Surgery",
    "Pediatrics-Minor Surgery",
    "Aerospace Medicine",
    "Bariatric-Surgery",
    "Colon And Rectal-Surgery",
    "Emergency Medical Technician (EMT)",
    "Hand-Surgery",
    "Forensic Medicine",
    "Geriatrics-No Surgery",
    "Hand-Surgery",
    "Nuclear Medicine",
    "O.R. Technician",
    "Occupational Medicine",
    "Pharmacist",
    "Phlebology",
    "Physicians NOC-No Surgery",
    "Physiotherapist",
    "Plastic Otorhinolaryngology-Surgery",
    "Respiratory Therapist",
    "Thoracic-Surgery",
    "Traumatic-Surgery",
    "Urgent Care-No Surgery"
]

In [33]:
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [6, 7, 8, 9],
    'colsample_bytree': [0.6, 0.65, 0.7, 0.75],
}

k_range = np.arange(50, 700, 100)
save_points_every = 5
speciality_pos = 11

results_test = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1", "Number of speciality Data Points", "Number of others Data Points", "Best Estimator"]]
results_train = [["Data Type", "Train / Test", "Speciality", "Precision 0", "Recall 0", "Precision 1", "Recall 1", "Macro Avg F1", "Number of speciality Data Points", "Number of others Data Points", "Best Estimator"]]
for spl in specialities_lis:
    for k in k_range:
        results_train, results_test = spit_train_test_split(X, y, spl, param_dist, results_test, results_train, k, tfidf=True)
    if speciality_pos%save_points_every == 0:
        df = pd.DataFrame(results_test + results_train)
        df.to_csv('./results_xgboost/binary_spl_prediction_tfidf_topk_output_v2_skewed_class_update_'+ str(speciality_pos/save_points_every) +'.csv', index=False)
    speciality_pos += 1
    print('\033[1m' + '\033[92m' + "Done: " + spl + '\033[0m' + '\033[0m')


Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Fitting 2 folds for each of 4 candidates, totalling 8 fits


...

Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Fitting 2 folds for each of 4 candidates, totalling 8 fits


c:\Users\ganga\anaconda3\envs\inferred_speciality\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


Done: Urgent Care-No Surgery


In [34]:
df.to_csv('./results_xgboost/binary_spl_prediction_tfidf_topk_output_v2_skewed_class_update_last.csv', index=False)